# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.10 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from ddpg_Agent import DDPGAGENT
import torch
import random
import numpy as np
import time

import matplotlib.pyplot as plt
from collections import deque


In [3]:
from unityagents import UnityEnvironment
import numpy as np
%matplotlib inline

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [6]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.0


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [7]:
def ddpg_control(num_episodes=500,max_t=2000,train_mode=True,print_every=1):
    ## mean scores from recent episodes
    scores_window = deque(maxlen=100)
    average = []
    mean = []
    max_ckpt = []
    min_ckpt = []
    
    for i_episode in range(1,num_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] #resetting the environment
        states = env_info.vector_observations # current state (observation)
        scores = np.zeros(num_agents)
        agent.reset()
        start_time = time.time()
        
        for t in range(max_t):
            actions = agent.act(states,noise_accumulate=True) #choosing action
            env_info = env.step(actions)[brain_name] #passing the actions into environment
            next_states = env_info.vector_observations #picking up the next state
            rewards = env_info.rewards # rewards you earn
            dones = env_info.local_done # notifying the status of the episode ( completed or not)
            
            for state,action,reward,next_state,done in zip(states,actions,rewards,next_states,dones):
                agent.step(state,action,reward,next_state,done,t)
            states = next_states
            scores += rewards
            
            if np.any(dones):
                break
        duration = time.time()-start_time #duration for an episode
        min_ckpt.append(np.min(scores)) #min score
        max_ckpt.append(np.max(scores)) #max score
        mean.append(np.mean(scores))    #mean score
        scores_window.append(mean[-1])  #appending to scores windoow
        average.append(np.mean(scores_window)) #average estimation
            
        if i_episode % print_every == 0 :
            print('\rEpisode {} ({} sec)  -- \tMin: {:.1f}\tMax: {:.1f}\tMean: {:.1f}\tAvg: {:.1f}'.format(\
                  i_episode, round(duration), min_ckpt[-1], max_ckpt[-1], mean[-1], average[-1]))
            
        if train_mode and mean[-1] > -np.Inf: # comparing with the best score (-np.Inf)
            torch.save(agent.actor_local.state_dict(), "ckpt_actor.pth") #if yes saving the learnable actor parameters
            torch.save(agent.critic_local.state_dict(),"ckpt_critic.pth")  #if yes saving the learnable critic parameters
                  
        if average[-1] >= 30.0 and i_episode >= 100: #compatibility with solvedscores
            print('\nEnvironment SOLVED in {} episodes!\tMoving Average ={:.1f} over last {} episodes'.format(\
                                        i_episode-100, average[-1], 100))
            
            if train_mode:
                torch.save(agent.actor_local.state_dict(), "ckpt_actor.pth") #if yes saving the learnable actor parameters
                torch.save(agent.critic_local.state_dict(), "ckpt_critic.pth")   #if yes saving the learnable critic parameters
            break
            
    return mean_scores, moving_avgs
                
            
            
        
    
    
    

In [8]:
agent = DDPGAGENT(state_size = state_size , action_size = action_size , random_seed = 1)
scores,avg = ddpg_control(num_episodes=500,max_t=1000)

Episode 1 (3 sec)  -- 	Min: 0.2	Max: 0.2	Mean: 0.2	Avg: 0.2
Episode 2 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.1
Episode 3 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.1
Episode 4 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 5 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 6 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 7 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 8 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 9 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 10 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 11 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 12 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 13 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 14 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 15 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 16 (3 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Avg: 0.0
Episode 17 (3 sec)  -- 	Min: 0.0	

KeyboardInterrupt: 

In [ ]:
#scores - visualization episode vs score
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label='DDPG')
plt.plot(np.arange(len(scores)), avg, c='r', label='moving avg')
plt.ylabel('Score')
plt.xlabel('Episode')
plt.show()

In [ ]:
env.close()